# ENTREGABLE 1

INSTRUCCIONES

Realizar la primera fase del análisis exploratorio (limpieza) usando el archivo CSV (dataset_banco.csv) con 45215 filas y 17 columnas.

# 1. El problema del negocio

Una entidad bancaria contrata a una empresa de marketing encargada de contactar telefónicamente a posibles clientes para determinar si están interesados o no en adquirir un certificado de depósito a término con el banco.

¿Qué perfil tienen los clientes con mayor potencial de conversión?

#2. El set de datos

Cada registro contiene 16 características (las primeras 16 columnas) y una categoría ("yes" o "no" dependiendo de si la persona está o no interesada en adquirir el producto). Las columnas son:

1. "age":  edad (numérica)
2. "job": tipo de trabajo (categórica: "admin.", "unknown", "unemployed", "management", "housemaid", "entrepreneur", "student", "blue-collar","self-employed", "retired", "technician", "services")
3. "marital": estado civil (categórica: "married", "divorced", "single")
4. "education": nivel educativo (categórica: "unknown", "secondary", "primary", "tertiary")
5. "default": si dejó de pagar sus obligaciones (categórica: "yes", "no")
6. "balance": saldo promedio anual en euros (numérica)
7. "housing": ¿tiene o no crédito hipotecario? (categórica: "yes", "no")
8. "loan": ¿tiene créditos de consumo? (categórica: "yes", "no")
9. "contact": medio a través del cual fue contactado (categórica: "unknown", "telephone", "cellular")
10. "day": último día del mes en el que fue contactada (numérica)
11. "month": último mes en el que fue contactada (categórica: "jan", "feb", "mar", ..., "nov", "dec")
12. "duration": duración (en segundos) del último contacto (numérica)
13. "campaign": número total de veces que fue contactada durante la campaña (numérica)
14. "pdays": número de días transcurridos después de haber sido contactado antes de la campaña actual (numérica. -1 indica que no fue contactado previamente)
15. "previous": número de veces que ha sido contactada antes de esta campaña (numérica)
16. "poutcome": resultado de la campaña de marketing anterior (categórica: "unknown", "other", "failure", "success")
17. "y": categoría ¿el cliente se suscribió a un depósito a término? (categórica: "yes", "no")

#3. Una primera mirada al dataset

In [1]:
# Importar librerías
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [15]:
# Lectura
ruta = "/content/dataset_banco.csv"
data = pd.read_csv(ruta)

In [16]:
print(data.shape)
data.head()

(12730, 17)


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143.0,yes,no,unknown,5.0,may,261.0,1.0,-1.0,0.0,unknown,no
1,44,technician,single,secondary,no,29.0,yes,no,unknown,5.0,may,151.0,1.0,-1.0,0.0,unknown,no
2,33,entrepreneur,married,secondary,no,2.0,yes,yes,unknown,5.0,may,76.0,1.0,-1.0,0.0,unknown,no
3,47,blue-collar,married,unknown,no,1506.0,yes,no,unknown,5.0,may,92.0,1.0,-1.0,0.0,unknown,no
4,33,unknown,single,unknown,no,1.0,no,no,unknown,5.0,may,198.0,1.0,-1.0,0.0,unknown,no


In [4]:
# Veamos las variables categóricas y las numéricas
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12740 entries, 0 to 12739
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   age        12740 non-null  int64  
 1   job        12740 non-null  object 
 2   marital    12740 non-null  object 
 3   education  12740 non-null  object 
 4   default    12739 non-null  object 
 5   balance    12739 non-null  float64
 6   housing    12739 non-null  object 
 7   loan       12739 non-null  object 
 8   contact    12739 non-null  object 
 9   day        12739 non-null  float64
 10  month      12739 non-null  object 
 11  duration   12739 non-null  float64
 12  campaign   12739 non-null  float64
 13  pdays      12739 non-null  float64
 14  previous   12739 non-null  float64
 15  poutcome   12739 non-null  object 
 16  y          12739 non-null  object 
dtypes: float64(6), int64(1), object(10)
memory usage: 1.7+ MB


In [5]:
# Generar estadísticas descriptivas para el DataFrame
data.describe()

,age,balance,day,duration,campaign,pdays,previous
count,12740.000000,12739.000000,12739.000000,12739.000000,12739.000000,12739.000000,12739.000000
mean,40.845055,1223.326478,15.291938,255.722113,2.831619,-0.985242,0.000392
std,12.091846,5356.779703,8.460270,256.823103,3.393142,1.665675,0.044300
min,20.000000,-3372.000000,1.000000,-1389.000000,1.000000,-1.000000,0.000000
25%,33.000000,48.000000,7.000000,104.000000,1.000000,-1.000000,0.000000
50%,39.000000,387.000000,15.000000,181.000000,2.000000,-1.000000,0.000000
75%,48.000000,1262.500000,21.000000,314.500000,3.000000,-1.000000,0.000000
max,530.000000,527532.000000,30.000000,3881.000000,63.000000,187.000000,5.000000


#4. Limpieza

Realizaremos el proceso de limpieza teniendo en cuenta las situaciones más comunes:

1. Datos faltantes en algunas celdas
2. Columnas irrelevantes (que no responden al problema que queremos resolver)
3. Registros (filas) repetidos
4. Valores extremos (*outliers*) en el caso de las variables numéricas. Se deben analizar en detalle pues no necesariamente la solución es eliminarlos
5. Errores tipográficos en el caso de las variables categóricas

Al final de este proceso de limpieza deberíamos tener un set de datos **íntegro**, listo para la fase de Análisis Exploratorio.

## 4.1 Datos faltantes

Comenzamos a ver que los datos no están completos, pues no todas las columnas tienen la misma cantidad de registros.

El número total de registros debería ser 45.215. Sin embargo columnas como "job", "marital", "education", "balance", "duration" y "pdays".

Por ser tan pocos los datos  faltantes optaremos por eliminar las filas correspondientes:

In [6]:
# Eliminar filas con datos faltantes en las columnas especificadas
data = data.dropna(subset=['job', 'marital', 'education', 'balance', 'duration', 'pdays'])

# Verificar la cantidad de registros después de eliminar las filas con datos faltantes
print("Número de registros después de eliminar filas con datos faltantes:", len(data))

print(data.shape)
data.head()

Número de registros después de eliminar filas con datos faltantes: 12739
(12739, 17)


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,no,2143.0,yes,no,unknown,5.0,may,261.0,1.0,-1.0,0.0,unknown,no
1,44,technician,single,secondary,no,29.0,yes,no,unknown,5.0,may,151.0,1.0,-1.0,0.0,unknown,no
2,33,entrepreneur,married,secondary,no,2.0,yes,yes,unknown,5.0,may,76.0,1.0,-1.0,0.0,unknown,no
3,47,blue-collar,married,unknown,no,1506.0,yes,no,unknown,5.0,may,92.0,1.0,-1.0,0.0,unknown,no
4,33,unknown,single,unknown,no,1.0,no,no,unknown,5.0,may,198.0,1.0,-1.0,0.0,unknown,no


##4.2 Columnas irrelevantes

Una columna irrelevante puede ser:

- **Una columna que no contiene información relevante para el problema que queremos resolver**. Por ejemplo en este caso podría ser una columna que no guarde relación con el posible perfil del cliente (deporte favorito, hobbies, comida favorita, etc.)
- **Una columna categórica pero con un sólo nivel**. Por ejemplo si en la columna "job" solo tuviésemos el nivel "unknown".
- **Una columna numérica pero con un sólo valor**. Por ejemplo si en la columna "edad" todos los vlaores fuesen iguales a 50.
- **Columnas con información redundante**. Por ejemplo si además de las columnas "month" y "day" tuviésemos la columna "month-day", resultado de combinar las dos anteriores.

Si tenemos la duda de si alguna columna puede ser relevante o no lo mejor es dejarla (y más adelante en posteriores etapas podremos darnos cuenta de si se debe preservar o no).

En este caso todas las columnas pueden resultar relevantes, pero debemos verificar que no haya columnas categóricas con un sólo nivel, o columnas numéricas con un sólo valor:

In [7]:
# Identificar columnas categóricas y numéricas:
categoric_columns = data.select_dtypes(include=['object']).columns
numeric_columns = data.select_dtypes(include=['int', 'float']).columns

# Verificamos si hay columnas categóricas con un solo nivel:
single_level_categorical = [col for col in categoric_columns if data[col].nunique() == 1]

# Vemos el resultado final
print("Columnas categóricas con un solo nivel:", single_level_categorical)

Columnas categóricas con un solo nivel: []


Todas las columnas categóricas tienen más de 1 subnivel. No eliminaremos ninguna.

Verifiquemos lo que ocurre con las columnas numéricas:

In [8]:
# Verificamos si hay columnas numéricas con un solo valor:
single_value_numeric = [col for col in numeric_columns if data[col].nunique() == 1]

# Vemos el resultado final
print("Columnas numéricas con un solo valor:", single_value_numeric)

Columnas numéricas con un solo valor: []


Todas las columnas numéricas tienen desviaciones estándar ("std") diferentes de cero, lo que indica que no tienen un único valor.

Preservaremos todas las columnas numéricas.

##4.3 Filas repetidas

In [9]:
# Eliminamos filas duplicadas del conjunto de datos
data = data.drop_duplicates()

# Verificamos el número de filas después de eliminar las filas duplicadas
print("Número de filas después de eliminar duplicados:", len(data))

Número de filas después de eliminar duplicados: 12738


##4.4 *Outliers* en las variables numéricas

No siempre se deben eliminar los *outliers* porque dependiendo de la variable numérica analizada estos pueden contener información importante.


In [11]:
# Filtramos las filas donde el valor de 'age' sea menor o igual a 110 y mayor o igual a 0
data = data[(data['age'] <= 100) & (data['age'] >= 0)]

# Filtramos las filas donde el valor de 'duration' sea mayor o igual a 0
data = data[data['duration'] >= 0]

# Filtramos las filas donde el valor de 'previous' sea menor o igual a 100
data = data[data['previous'] <= 100]

print(data.head())
print()
print("Número de filas después de eliminar outliers:", len(data))

   age           job  marital  education default  balance housing loan  \
0   58    management  married   tertiary      no   2143.0     yes   no   
1   44    technician   single  secondary      no     29.0     yes   no   
2   33  entrepreneur  married  secondary      no      2.0     yes  yes   
3   47   blue-collar  married    unknown      no   1506.0     yes   no   
4   33       unknown   single    unknown      no      1.0      no   no   

   contact  day month  duration  campaign  pdays  previous poutcome   y  
0  unknown  5.0   may     261.0       1.0   -1.0       0.0  unknown  no  
1  unknown  5.0   may     151.0       1.0   -1.0       0.0  unknown  no  
2  unknown  5.0   may      76.0       1.0   -1.0       0.0  unknown  no  
3  unknown  5.0   may      92.0       1.0   -1.0       0.0  unknown  no  
4  unknown  5.0   may     198.0       1.0   -1.0       0.0  unknown  no  

Número de filas después de eliminar outliers: 12730


##4.5 Errores tipográficos en variables categóricas

En una variable categórica pueden aparecer sub-niveles como "unknown" y "UNK" que para nosotros son equivalentes pero que para nuestro programa parecerían diferentes.

Se deben unificar estos sub-niveles

In [12]:
# Iteramos sobre todas las columnas del DataFrame para obener los valores únicos de cada columna
for column in data.columns:
    unique_values = data[column].unique()
    print(f"Valores únicos de {column}: {unique_values}")

Valores únicos de age: [58 44 33 47 35 28 42 43 41 29 57 51 45 60 56 32 25 40 39 52 46 36 49 59
 53 37 50 54 55 48 24 38 31 30 27 34 23 26 61 22 21 20]
Valores únicos de job: ['management' 'technician' 'entrepreneur' 'blue-collar' 'unknown'
 'Management' 'retired' 'admin.' 'services' 'self-employed' 'MANAGEMENT'
 'Self-employed' 'unemployed' 'housemaid' 'student' 'Services' 'Retired'
 'administrative']
Valores únicos de marital: ['married' 'single' 'div.' 'divorced' 'DIVORCED' 'Single']
Valores únicos de education: ['tertiary' 'secondary' 'unknown' 'primary' 'SECONDARY' 'Secondary'
 'Primary' 'sec.' 'Tertiary' 'UNK']
Valores únicos de default: ['no' 'yes']
Valores únicos de balance: [2.143e+03 2.900e+01 2.000e+00 ... 1.444e+03 3.531e+03 2.220e+03]
Valores únicos de housing: ['yes' 'no']
Valores únicos de loan: ['no' 'yes' 'No' 'YES' 'Yes' 'NO']
Valores únicos de contact: ['unknown' 'cellular' 'telephone']
Valores únicos de day: [ 5.  6.  8.  7. 16.  9. 12. 13. 14. 15. 19. 20. 21. 23. 2

In [13]:
data = data.replace('UNK', 'unknown')
data['poutcome'] = data['poutcome'].replace('Success', 'success')
data['job'] = data['job'].replace('MANAGEMENT', 'management')
data['job'] = data['job'].replace('Management', 'management')
data['job'] = data['job'].replace('Self-employed', 'self-employed')
data['job'] = data['job'].replace('Retired', 'retired')
data['job'] = data['job'].replace('admin.', 'administrative')
data['job'] = data['job'].replace('Services', 'services')
data['loan'] = data['loan'].replace('YES', 'yes')
data['loan'] = data['loan'].replace('Yes', 'yes')
data['loan'] = data['loan'].replace('NO', 'no')
data['loan'] = data['loan'].replace('No', 'no')
data['contact'] = data['contact'].replace('phone', 'telephone')
data['contact'] = data['contact'].replace('mobile', 'cellular')
data['marital'] = data['marital'].replace('DIVORCED', 'divorced')
data['marital'] = data['marital'].replace('div.', 'divorced')
data['marital'] = data['marital'].replace('Single', 'single')
data['education'] = data['education'].replace('SECONDARY', 'secondary')
data['education'] = data['education'].replace('sec.', 'secondary')
data['education'] = data['education'].replace('Secondary', 'secondary')
data['education'] = data['education'].replace('Primary', 'primary')
data['education'] = data['education'].replace('Tertiary', 'tertiary')

## Conclusiones

In [14]:
data.to_csv(ruta, index=False)

Hemos llevado a cabo la primera fase del análisis exploratorio (limpieza) usando un archivo CSV (dataset_banco.csv).

Para llevar a cabo la limpieza lo hemos hecho en varias partes: primero hemos eliminado algunas filas de datos faltantes, ya que eran pocos. Después hemos decidido suprimir algunas columnas que nos resultavan irrelevantes como pueden ser las columnas categóricas con un sólo nivel, o columnas numéricas con un sólo valor, pero no había ninguna. Luego hemos eliminado algunas filas que estaban duplicadas y algunos outliers que hemos considerado que eran datos erróneos.

Por último, hemos modificado algunos valores de las variables categóricas, ya que hemos encontrado sub-niveles que hacían que palabras que deberían ser las mismas nos contaran y aparecieran como diferentes. Dejando de esta manera nuestro dataset ya limpio y listo para la siguiente fase del análisis exploratorio (análisis descriptivo y visualización de datos).

